In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd

# Store items in Filipino-English DF

# Filipino lines
with open('en-fil.txt/QED.en-fil.fil', 'r') as file:
    fil_lines = file.readlines()

# English lines
with open('en-fil.txt/QED.en-fil.en', 'r') as file:
    en_lines = file.readlines()

combined_items = list(zip(en_lines, fil_lines))
df = pd.DataFrame(combined_items, columns=['English', 'Filipino'])
print(len(df))

42864


In [3]:
from sklearn.model_selection import train_test_split

# Retrieve some matches for fine tuning
df['Split'] = 'unset'
for_translate, for_shots = train_test_split(df, test_size=0.1, random_state=42)
train, test = train_test_split(for_translate, test_size=0.1, random_state=42)

df.loc[for_shots.index, 'Split'] = 'shots'
df.loc[train.index, 'Split'] = 'train'
df.loc[test.index, 'Split'] = 'test'

In [4]:
!pip install transformers

In [5]:
from transformers import LlamaConfig, LlamaForCausalLM


config_path = "config.json"
config = LlamaConfig.from_json_file(config_path)

model = LlamaForCausalLM(config)

print(model)

/data/students/juan/anaconda3/envs/ai351/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm)

In [6]:
!pip install datasets safetensors accelerate

In [7]:
from datasets import Dataset, DatasetDict

# Convert your DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
print(dataset)

dataset_dict = DatasetDict({
    "train": dataset.filter(lambda x: x["Split"] == "train"),
    "test": dataset.filter(lambda x: x["Split"] == "test")
})

# Access train and validation datasets
train_dataset = dataset_dict["train"]
val_dataset = dataset_dict["test"]


Dataset({
    features: ['English', 'Filipino', 'Split'],
    num_rows: 42864
})


Filter: 100%|██████████| 42864/42864 [00:00<00:00, 268197.99 examples/s]


In [8]:
!pip install sentencepiece safetensors
!pip install -U "huggingface_hub[cli]"

In [10]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

save_dir = "./models/CuatroLLM"

os.makedirs(save_dir, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained("britllm/CuatroLLM")
tokenizer.save_pretrained(save_dir)

model = AutoModelForCausalLM.from_pretrained('britllm/CuatroLLM')
model.save_pretrained(save_dir)

tokenizer.pad_token = tokenizer.eos_token

In [11]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["English"],
        text_target=examples["Filipino"],
        padding="max_length",
        truncation=True,
        max_length=1024,
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 3858/3858 [00:01<00:00, 1949.62 examples/s]


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,        # Reduce batch size
    gradient_accumulation_steps=8,       # Accumulate gradients over 8 steps
    num_train_epochs=3,
    fp16=True,                            # Use mixed precision
    save_steps=500,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    optim="adamw_torch",
)


/data/students/juan/anaconda3/envs/ai351/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)


/tmp/ipykernel_1843522/1103220103.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [14]:
trainer.train()

/data/students/juan/anaconda3/envs/ai351/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 471.38 MiB is free. Including non-PyTorch memory, this process has 38.91 GiB memory in use. Of the allocated memory 33.96 GiB is allocated by PyTorch, and 3.49 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)